In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../modules'))

In [3]:
import pandas as pd
import os

from bokeh.plotting import figure, ColumnDataSource, output_file, show, save, output_file, output_notebook
from bokeh.layouts import column, row, gridplot
from bokeh.models import BooleanFilter, CDSView, Select, Range1d
from bokeh.models import HoverTool,PanTool,ResetTool,HoverTool,WheelZoomTool,SaveTool,BoxZoomTool,UndoTool,RedoTool,SaveTool,ZoomInTool,ZoomOutTool
from bokeh.models.widgets import Dropdown
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.models.transforms import CustomJSTransform
from bokeh.palettes import Category20
from bokeh.io import curdoc

output_notebook()

Loading BokehJS ...

In [4]:
import own.funny as funny

import own.data.flights as Flights
import own.data.gas as Gas
import own.data.oil as Oil

import own.data.stocks2 as Stocks

import own.data.ecdc as ECDC
import own.data.countries as Countries

from own.colors import focus_countries

In [5]:
oil = Oil.data_merged
gas = Gas.data
flights = Flights.combined()

stocks = Stocks.close_values()

stocks_normalized = stocks.div(stocks.loc['2020-01-02'])

deaths = ECDC.totals()['deaths_total'].unstack()
deaths_sum = deaths.sum(axis=1)
deaths_sum.name = 'deaths_sum'

countries_names = Countries.country_names()['common_name']

In [6]:
oil_source = ColumnDataSource(oil)
gas_source = ColumnDataSource(gas)
flights_source = ColumnDataSource(flights)

stocks_source = ColumnDataSource(stocks)
stocks_normalized_source = ColumnDataSource(stocks_normalized)

In [7]:
def TOOLS():
    return [
#       HoverTool(),
        BoxZoomTool(dimensions='both'),
        PanTool(dimensions='width'),
        ZoomInTool(),
        ZoomOutTool(),
        ResetTool(),
        SaveTool(),
        UndoTool(),
        RedoTool(),
    ]

In [8]:
from bokeh.models import LinearAxis, Range1d

import datetime
    
def oil_plot():
    deaths_source = ColumnDataSource(deaths_sum.to_frame())

    p = figure(
        plot_height = 200, plot_width = 466,

        x_range = [
            datetime.datetime(2019, 7, 1).timestamp() * 1000,
            datetime.datetime.today().timestamp() * 1000,
        ],
        y_range=[0,80],

        x_axis_type='datetime',
        title = 'Daily fossil fuels prices',
        x_axis_label='Date',
        y_axis_label='Price ($)',

        tools=TOOLS(),
        toolbar_location='right',

        extra_y_ranges = {
            'secondary_y': Range1d(
                start = 0,
                end = deaths_sum.max()
            )
        },
    )
    p.add_layout(
        LinearAxis(
            y_range_name = "secondary_y", 
            axis_label = 'deaths (worldwide)'
        ),
        'right'
    )
    p.background_fill_color = '#f0f0f0'
    p.background_fill_alpha = 0.20

    p.xaxis.major_label_text_font_size = '11pt'
    #p.xaxis.major_label_text_font_style = 'bold'

    p.yaxis[0].formatter = NumeralTickFormatter(format="$ 1,000")
    p.yaxis[1].formatter = NumeralTickFormatter(format="1,000")
    
    oil_lines = {
        'average': p.line(source = oil_source, x='Date', y='Average', color='#990000', line_width=3,   alpha=1.00, muted_alpha=0.1, legend_label='Oil (Average)', name='Oil (Average)'),
        'breant':  p.line(source = oil_source, x='Date', y='Breant',  color='#009900', line_width=1.5, alpha=0.67, muted_alpha=0.1, legend_label='Oil (Brent)',   name='Oil (Brent)'),
        'twi':     p.line(source = oil_source, x='Date', y='Twi',     color='#999900', line_width=1.5, alpha=0.67, muted_alpha=0.1, legend_label='Oil (Twi)',     name='Oil (Twi)'),
    }
    oil_lines['breant'].muted = True
    oil_lines['twi'].muted = True

    gas_line = p.line(source = gas_source, x='Date', y='Price', color='#0000cc', line_width=3, alpha=1.00, muted_alpha=0.1, legend_label='Natural Gas', name="Natural Gas")
    
    corona_line = p.line(source = deaths_source, x='date', y='deaths_sum',     color='#666666', line_width=2.5, alpha=1.00, muted_alpha=0.1, legend_label='Deaths (worldwide)', name="Corona Deaths", y_range_name='secondary_y')

    
    p.line(
        [
            datetime.datetime(2020, 4, 1).timestamp() * 1000,
            datetime.datetime(2020, 4, 1).timestamp() * 1000
        ],
        [0, 80],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'dashed'
    )
    p.line(
        [
            datetime.datetime(2020, 1, 1).timestamp() * 1000,
            datetime.datetime(2020, 1, 1).timestamp() * 1000
        ],
        [0, 80],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'solid'
    )
    p.line(
        [
            datetime.datetime(2019, 10, 1).timestamp() * 1000,
            datetime.datetime(2019, 10, 1).timestamp() * 1000
        ],
        [0, 80],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'dashed'
    )
    p.line(
        [
            datetime.datetime(2019, 7, 1).timestamp() * 1000,
            datetime.datetime(2019, 7, 1).timestamp() * 1000
        ],
        [0, 80],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'solid'
    )
    
    p.legend.location = "bottom_left"

    p.legend.background_fill_color = '#ffffff'
    p.legend.background_fill_alpha = 0.75
    p.legend.border_line_alpha = 0.75

    p.legend.click_policy = "mute"

    p.toolbar.logo = None
    
    p.xgrid.visible = False
    
    p.add_tools(HoverTool(
        renderers=[ 
            oil_lines['average'], 
            oil_lines['breant'], 
            oil_lines['twi'], 
            gas_line 
        ], 
        tooltips = [
            ('', "$name"),
            ("Date", "$x{%Y-%m-%d}"),
            ("Price", '$y{0.00}'),
        ],
        formatters = {
            '$x': 'datetime',
        },
    ))
    p.add_tools(HoverTool(
        renderers=[ corona_line ], 
        tooltips = [
            ('', "$name"),
            ("Date", "$x{%Y-%m-%d}"),
            ("Deaths", '$y{0,000}'),
        ],
        formatters = {
            '$x': 'datetime',
        },
    ))

    return p

In [9]:
plot = oil_plot()

grid = gridplot(
    [
        plot,
    ],
    ncols=1,
    sizing_mode='scale_both',
    merge_tools = True,
    toolbar_options = dict(logo=None),
    toolbar_location = 'right'
)

if funny.is_save_for_later():
    funny.save_for_later('economy-fossile', grid)

if funny.is_save_to_public():
    output_file('../docs/plots/economy-fossile.html', mode='inline')#, mode='relative-dev', root_dir='../lib/bokeh/2.0.1')
    save(grid)

if funny.is_display():
    output_notebook()
    show(grid)

In [10]:
from bokeh.models import LinearAxis, Range1d

import datetime

def flights_plot():
    deaths_source = ColumnDataSource(deaths_sum.to_frame())

    p = figure(
        plot_height = 200, plot_width = 466,

        x_range = [
            datetime.datetime(2019, 12, 30).timestamp() * 1000,
            datetime.datetime.today().timestamp() * 1000,
        ],
        y_range=[-5,200000],


        x_axis_type='datetime',
        title = 'Daily and 7 day average of flight data (January-May 2020)',
        x_axis_label='Date',
        y_axis_label='Tracked flights',

        tools=TOOLS(),
        toolbar_location='right',

        extra_y_ranges = {
            'secondary_y': Range1d(
                start = 0,
                end = deaths_sum.max()
            )
        },
    )
    p.add_layout(
        LinearAxis(
            y_range_name = "secondary_y", 
            axis_label = 'deaths (worldwide)'
        ),
        'right'
    )

    p.background_fill_color = '#f0f0f0'
    p.background_fill_alpha = 0.20

    p.xaxis.major_label_text_font_size = '12pt'
    #p.xaxis.major_label_text_font_style = 'bold'

    p.yaxis[0].formatter = NumeralTickFormatter(format="1,000")
    p.yaxis[1].formatter = NumeralTickFormatter(format="1,000")

    flights_lines = {
        'total7':      p.line(source = flights_source, x='date', y='total_7day_average',      color='#0000cc', muted_color='#666666', line_width=3.0, alpha=1.0, muted_alpha=0.2, legend_label='Total planes (7-day average)',      name='Total planes (7-day average)'),
        'total':       p.line(source = flights_source, x='date', y='total',                   color='#000099', muted_color='#666666', line_width=1.5, alpha=0.5, muted_alpha=0.2, legend_label='Total planes (daily)',              name='Total planes (daily)', muted=True),
        'commercial7': p.line(source = flights_source, x='date', y='commercial_7day_average', color='#cc0000', muted_color='#666666', line_width=3.0, alpha=1.0, muted_alpha=0.2, legend_label='Commercial planes (7-day average)', name='Commercial planes (7-day average)'),
        'commercial':  p.line(source = flights_source, x='date', y='commercial',              color='#990000', muted_color='#666666', line_width=1.5, alpha=0.5, muted_alpha=0.2, legend_label='Commercial planes (daily)',         name='Commercial planes (daily)', muted=True),
    }
    
    corona_line = p.line(source = deaths_source, x='date', y='deaths_sum',     color='#666666', line_width=2.5, alpha=1.00, muted_alpha=0.1, legend_label='Deaths (worldwide)', y_range_name='secondary_y', name="Corona Deaths")

    p.line(
        [
            datetime.datetime(2020, 4, 1).timestamp() * 1000,
            datetime.datetime(2020, 4, 1).timestamp() * 1000
        ],
        [0, 200000],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'dashed'
    )
    p.line(
        [
            datetime.datetime(2020, 1, 1).timestamp() * 1000,
            datetime.datetime(2020, 1, 1).timestamp() * 1000
        ],
        [0, 200000],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'solid'
    )

    p.legend.location = "bottom_left"

    p.legend.background_fill_color = '#ffffff'
    p.legend.background_fill_alpha = 0.75
    p.legend.border_line_alpha = 0.75

    p.legend.click_policy = "mute"

    p.toolbar.logo = None
    
    p.xgrid.visible = False
    
    p.add_tools(HoverTool(
        renderers=[
            flights_lines['total7'],
            flights_lines['total'],
            flights_lines['commercial7'],
            flights_lines['commercial']
        ], 
        tooltips = [
            ('', "$name"),
            ("Date", "$x{%Y-%m-%d}"),
            ("Plains", '$y{0,000}'),
        ],
        formatters = {
            '$x': 'datetime',
        },
    ))
    p.add_tools(HoverTool(renderers=[ corona_line ], 
        tooltips = [
            ('', "$name"),
            ("Date", "$x{%Y-%m-%d}"),
            ("Deaths", '$y{0,000}'),
        ],
        formatters = {
            '$x': 'datetime',
        },
    ))

    return p

In [11]:
plot = flights_plot()

grid = gridplot(
    [
        plot,
    ],
    ncols=1,
    sizing_mode='scale_both',
    merge_tools = True,
    toolbar_options = dict(logo=None),
    toolbar_location = 'right'
)

if funny.is_save_for_later():
    funny.save_for_later('economy-flights', grid)

if funny.is_save_to_public():
    output_file('../docs/plots/economy-flights.html', mode='inline')#, mode='relative-dev', root_dir='../lib/bokeh/2.0.1')
    save(grid)

if funny.is_display():
    output_notebook()
    show(grid)

In [12]:
def TOOLS():
    return [
#       HoverTool(),
        BoxZoomTool(dimensions='both'),
        PanTool(dimensions='both'),
        ZoomInTool(),
        ZoomOutTool(),
        ResetTool(),
        SaveTool(),
        UndoTool(),
        RedoTool(),
    ]

In [13]:
from bokeh.models import LinearAxis, Range1d

import datetime

def stocks_plot(source, y_range, left_y_formater, title, left_y_axis_label, is_norm = False):
    deaths_source = ColumnDataSource(deaths_sum.to_frame())
    
    p = figure(
        plot_height = 200, plot_width = 355,

        x_range = [
            datetime.datetime(2019, 12, 30).timestamp() * 1000,
            datetime.datetime.today().timestamp() * 1000,
        ],
        y_range=y_range,

        x_axis_type='datetime',

        x_axis_label='Date',
        y_axis_label=left_y_axis_label,

        tools=TOOLS(),
        toolbar_location='below',

        title = title,
        
        extra_y_ranges = {
            'secondary_y': Range1d(
                start = 0,
                end = deaths_sum.max()
            )
        },
    )
    p.add_layout(
        LinearAxis(
            y_range_name = "secondary_y", 
            axis_label = 'deaths (worldwide)'
        ),
        'right'
    )

    p.background_fill_color = '#f0f0f0'
    p.background_fill_alpha = 0.20

    p.xaxis.major_label_text_font_size = '12pt'
    #p.xaxis.major_label_text_font_style = 'bold'

    p.yaxis[0].formatter = NumeralTickFormatter(format=left_y_formater)
    p.yaxis[1].formatter = NumeralTickFormatter(format="1,000")

    
    stock_lines = {}
    
    for country in stocks.columns:
        stock_lines[country] = p.line(source = source, x='date', y=country, color=focus_countries[country], muted_color='#666666', line_width=2.25, alpha=1.0, muted_alpha=0.2, legend_label=countries_names[country], name=countries_names[country]),
    
    corona_line = p.line(source = deaths_source, x='date', y='deaths_sum', color='#666666', line_width=2.5, alpha=1.00, muted_alpha=0.1, legend_label='Deaths (worldwide)', y_range_name='secondary_y', name="Corona Deaths")

    p.line(
        [
            datetime.datetime(2020, 4, 1).timestamp() * 1000,
            datetime.datetime(2020, 4, 1).timestamp() * 1000
        ],
        [0, y_range[1]],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'dashed'
    )
    p.line(
        [
            datetime.datetime(2020, 1, 1).timestamp() * 1000,
            datetime.datetime(2020, 1, 1).timestamp() * 1000
        ],
        [0, y_range[1]],
        line_color = 'gray',
        line_alpha = 0.5,
        line_width = 2,
        line_dash = 'solid'
    )

    p.legend.location = "bottom_left"

    p.legend.background_fill_color = '#ffffff'
    p.legend.background_fill_alpha = 0.75
    p.legend.border_line_alpha = 0.75

    p.legend.click_policy = "mute"

    p.toolbar.logo = None
    
    p.xgrid.visible = False
    if is_norm:
        p.add_tools(HoverTool(
            tooltips = [
                ('', "$name"),
                ("Date", "$x{%Y-%m-%d}"),
                ("Value", '$y'),

            ],
            formatters = {
                '$x': 'datetime',
            },
        ))
    else:
        p.add_tools(HoverTool(
            tooltips = [
                ('', "$name"),
                ("Date", "$x{%Y-%m-%d}"),
                ("Value", '$y{0,000}'),

            ],
            formatters = {
                '$x': 'datetime',
            },
        ))
    p.add_tools(HoverTool(renderers=[ corona_line ], 
        tooltips = [
            ('', "$name"),
            ("Date", "$x{%Y-%m-%d}"),
            ("Deaths", '$y{0,000}'),
        ],
        formatters = {
            '$x': 'datetime',
        },
    ))


    return p

In [14]:
plot = stocks_plot(stocks_source, [0, 25000], "1,000", 'Stock values of indexes in focus countries', 'Stock index value')

grid = gridplot(
    [
        plot,
    ],
    ncols=1,
    sizing_mode='scale_both',
    merge_tools = True,
    toolbar_options = dict(logo=None),
    toolbar_location = 'right'
)

if funny.is_save_for_later():
    funny.save_for_later('economy-stocks', grid)

if funny.is_save_to_public():
    output_file('../docs/plots/economy-stocks.html', mode='inline')#, mode='relative-dev', root_dir='../lib/bokeh/2.0.1')
    save(grid)

if funny.is_display():
    output_notebook()
    show(grid)

In [15]:
plot = stocks_plot(stocks_normalized_source, [0.6,1.15], "0 %", 'Normalized stock values of indexes in focus countries', 'Stock index value (normalized)', True)

grid = gridplot(
    [
        plot,
    ],
    ncols=1,
    sizing_mode='scale_both',
    merge_tools = True,
    toolbar_options = dict(logo=None),
    toolbar_location = 'right'
)

if funny.is_save_for_later():
    funny.save_for_later('economy-stocks-normalized', grid)

if funny.is_save_to_public():
    output_file('../docs/plots/economy-stocks-normalized.html', mode='inline')#, mode='relative-dev', root_dir='../lib/bokeh/2.0.1')
    save(grid)

if funny.is_display():
    output_notebook()
    show(grid)    